In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import time


In [ ]:
# Path to your video in Google Drive
video_path = "/content/drive/MyDrive/Drone view of Rawalpindi #travelwithnadir #drone #rawalpindi.mp4"

# Folder to save processed frames
save_dir = "/content/drive/MyDrive/UAV_Project/clean_frames"

# Create folder if not exists
os.makedirs(save_dir, exist_ok=True)

print("Video Path:", video_path)
print("Output Folder:", save_dir)


Video Path: /content/drive/MyDrive/Drone view of Rawalpindi #travelwithnadir #drone #rawalpindi.mp4
Output Folder: /content/drive/MyDrive/UAV_Project/clean_frames


In [ ]:
# Cell A: check mount and list top-level MyDrive
import os
print("Mounted path exists:", os.path.exists('/content/drive'))
print("Mounted MyDrive exists:", os.path.exists('/content/drive/MyDrive'))
print("List /content/drive (top-level):")
!ls -la /content/drive
print("\nList /content/drive/MyDrive (first 50 entries):")
!ls -la "/content/drive/MyDrive" | sed -n '1,50p'


Mounted path exists: True
Mounted MyDrive exists: True
List /content/drive (top-level):
total 16
dr-x------ 4 root root 4096 Dec  6 05:59 .Encrypted
drwx------ 7 root root 4096 Dec  6 05:59 MyDrive
dr-x------ 2 root root 4096 Dec  6 05:59 .shortcut-targets-by-id
drwx------ 5 root root 4096 Dec  6 05:59 .Trash-0

List /content/drive/MyDrive (first 50 entries):
total 176432
-rw------- 1 root root    59037 Nov  5  2024 04162213007(bootstrap).zip
-rw------- 1 root root   749834 Oct 21  2024 A1_WEB_ZIP.zip
drwx------ 2 root root     4096 Jan 21  2024 ahsan economics(04162213007)
-rw------- 1 root root    62541 Oct 20 12:38 ASSIGNMENT1(04162213007) (1).docx
-rw------- 1 root root    62541 Oct 20 12:38 ASSIGNMENT1(04162213007).docx
-rw------- 1 root root      184 Jul 14 17:24 Class 1     urdu.gdoc
drwx------ 2 root root     4096 Mar 10  2025 Classroom
drwx------ 2 root root     4096 Mar 11  2025 Colab Notebooks
-rw------- 1 root root     6669 May 14  2025 ConsoleApplication1-1.cpp
-rw------- 

In [ ]:
# Cell C: run shell find to search for common video extensions
!find /content/drive/MyDrive -maxdepth 4 -type f \( -iname "*.mp4" -o -iname "*.mov" -o -iname "*.avi" -o -iname "*.mkv" \) -print | sed -n '1,200p'


/content/drive/MyDrive/Drone view of Rawalpindi #travelwithnadir #drone #rawalpindi.mp4


In [ ]:
video_path = "/content/drive/MyDrive/Drone view of Rawalpindi #travelwithnadir #drone #rawalpindi.mp4"   # <-- CHANGE THIS

import os
print("Exists?" , os.path.exists(video_path))


Exists? True


In [ ]:
start_time = time.time()   # start timer

# ---------------- SETTINGS ---------------- #
target_size = (720, 720)         # resize all frames
blur_threshold = 140             # blur detection threshold
brightness_min = 60              # too dark
brightness_max = 200             # too bright
# -------------------------------------------- #

# Load video
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Extract 3 frames per second
extract_every = max(1, int(fps / 3))

print("FPS:", fps)
print("Total Frames:", frame_count)
print("Extracting every", extract_every, "frames.")

# ---- Define blur checking ---- #
def is_blurry(image, threshold=blur_threshold):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < threshold, variance

i = 0
saved = 0

with tqdm(total=frame_count) as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract every N frames
        if i % extract_every == 0:

            # ----- Blur test -----
            blurry, blur_score = is_blurry(frame)
            if blurry:
                i += 1
                pbar.update(1)
                continue

            # ----- Brightness test -----
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            brightness = gray.mean()

            if brightness < brightness_min or brightness > brightness_max:
                i += 1
                pbar.update(1)
                continue

            # ----- Resize -----
            frame = cv2.resize(frame, target_size)

            # ----- Save -----
            filename = f"frame_{i}_blur{int(blur_score)}_light{int(brightness)}.jpg"
            cv2.imwrite(os.path.join(save_dir, filename), frame)

            saved += 1

        i += 1
        pbar.update(1)

cap.release()

end_time = time.time()
elapsed = end_time - start_time

print("===================================")
print("DONE!")
print("Frames Saved:", saved)
print("Saved to:", save_dir)
print("Time Taken:", round(elapsed, 2), "seconds")
print("===================================")


FPS: 30.0
Total Frames: 606
Extracting every 10 frames.


100%|█████████▉| 603/606 [00:32<00:00, 18.84it/s]

DONE!
Frames Saved: 61
Saved to: /content/drive/MyDrive/UAV_Project/clean_frames
Time Taken: 32.82 seconds


In [ ]:
import numpy as np
import glob
import os
from tqdm import tqdm

# Example camera parameters (replace with yours)
K = np.array([[1200, 0, 360],
              [0, 1200, 360],
              [0, 0, 1]], dtype=np.float32)
D = np.array([-0.25, 0.1, 0, 0], dtype=np.float32)

input_dir = save_dir  # frames saved from your code
undistorted_dir = os.path.join(save_dir, "undistorted")
os.makedirs(undistorted_dir, exist_ok=True)

frame_paths = glob.glob(os.path.join(input_dir, "*.jpg"))

for path in tqdm(frame_paths, desc="Undistorting frames"):
    img = cv2.imread(path)
    h, w = img.shape[:2]
    new_K, _ = cv2.getOptimalNewCameraMatrix(K, D, (w, h), 1)
    undistorted = cv2.undistort(img, K, D, None, new_K)
    cv2.imwrite(os.path.join(undistorted_dir, os.path.basename(path)), undistorted)

print("Lens distortion correction done!")


Undistorting frames: 100%|██████████| 61/61 [00:31<00:00,  1.92it/s]

Lens distortion correction done!


In [ ]:
import rasterio
from rasterio.transform import from_origin
from pyproj import Transformer

# Example UAV metadata (replace with real data)
uav_lat = 33.7470
uav_lon = 73.1380
uav_alt_m = 600 # UAV height above ground in meters

sensor_width_mm = 6.4
focal_length_mm = 4.5
image_width_px = target_size[0]
image_height_px = target_size[1]

# Compute ground sample distance (GSD) in meters per pixel
GSD = (uav_alt_m * sensor_width_mm) / (focal_length_mm * image_width_px)
print("GSD:", GSD, "m/pixel")

# Coordinate transformer (WGS84 -> Web Mercator)
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
uav_x, uav_y = transformer.transform(uav_lon, uav_lat)

ortho_dir = os.path.join(save_dir, "orthorectified")
os.makedirs(ortho_dir, exist_ok=True)

for path in tqdm(glob.glob(os.path.join(undistorted_dir, "*.jpg")), desc="Orthorectifying"):
    img = cv2.imread(path)

    transform = from_origin(uav_x - (img.shape[1]/2)*GSD,
                            uav_y + (img.shape[0]/2)*GSD,
                            GSD, GSD)

    with rasterio.open(
        os.path.join(ortho_dir, os.path.basename(path).replace(".jpg", ".tif")),
        "w",
        driver="GTiff",
        height=img.shape[0],
        width=img.shape[1],
        count=3,
        dtype=img.dtype,
        transform=transform,
        crs="EPSG:3857"
    ) as dst:
        dst.write(img[:, :, 0], 1)
        dst.write(img[:, :, 1], 2)
        dst.write(img[:, :, 2], 3)

print("Orthorectification complete!")


GSD: 1.1851851851851851 m/pixel


Orthorectifying: 100%|██████████| 61/61 [00:37<00:00,  1.61it/s]

Orthorectification complete!


In [ ]:
import cv2
import os
import glob
from tqdm import tqdm

patch_size = 256

# The orthorectified images directory
ortho_dir = os.path.join(save_dir, "orthorectified_frames")

# Where patches will be saved
patch_dir = os.path.join(save_dir, "patches")
os.makedirs(patch_dir, exist_ok=True)

print("Patching from ORTHO frames...")

for path in tqdm(glob.glob(os.path.join(ortho_dir, "*.tif")), desc="Creating patches"):
    img = cv2.imread(path)

    if img is None:
        print(f"Warning: Could not read {path}")
        continue

    h, w = img.shape[:2]

    # Slide patch window over image
    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):

            patch = img[y:y+patch_size, x:x+patch_size]

            # Skip incomplete boundary patches
            if patch.shape[0] != patch_size or patch.shape[1] != patch_size:
                continue

            patch_name = os.path.basename(path).replace(".tif", f"_patch_{x}_{y}.jpg")
            cv2.imwrite(os.path.join(patch_dir, patch_name), patch)

print("Patches created successfully!")


Patching from ORTHO frames...


Creating patches: 0it [00:00, ?it/s]

Patches created successfully!
